In [ ]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

# set kinetic parameters
with open("rate_parameters.json") as infile:
    jsdata = json.load(infile)

params = jsdata["kin1"]

Copyright **Jacob Martin and Paolo Raiteri**, January 2021

## Numerical solution of chemical equilibrium problems #1
Imagine a simple dimerisation reaction
\begin{equation}
2A \to B
\end{equation}

whose equilibrium constant can be written as
\begin{equation}
K_{eq} = \frac{[B]}{[A]^2} = 0.156
\end{equation}

and wanting to calculate the equilibrium concentrations of $[A]_{eq}$ and $[B]_{eq}$ given their initial concentrations $[A]_{0}$ and $[B]_{0}$.
Although this is a simple problem that can be solved analytically, in this workshop we will learn how we can use an iterative method to numerically solve it.
We will use a relatively simple minimisation procedure that can be applied to a large number of problems, for which it is not possible or it is too complicated to get an analytic solution.

Imagine mixing the reagents and then to be able to monitor the concentration of all the species in the system at discrete time intervals (*timesteps*). What you will see is that the concentrations will change and tend to the equilibrium value. As you have learnt in first year, the reaction quotient, $Q$, can be used to decided which way to reaction will proceed, and that at equilibrium the reaction quotient is equal to the equilibrium constant. Hence, as we have discussed in class, the reaction quotient and the equilibrium constant can be use to define a *driving force* that pulls the system towards equilibrium.

This *driving force* can then be used in conjunction with an *ICE* table to numerically compute the equilibrium concentration of reactant and products.

|      | [A]        | [B]
| :--- | :--------: |:---------:
| *I*  | [A]$_0$    | [B]$_0$
| *C*  | -2x        | x
| *E*  | [A]$_0$-2x | [B]$_0$+x

Here below you can see the working principle of the minimisation procedure that we will emply

1. compute the reaction quotient at beginning of the experiment
\begin{equation}
Q = \dfrac{\mathrm{[B]}_0}{\mathrm{[A]}^2_0}
\end{equation}

2. compute the driving force.

\begin{equation}
F \propto \ln\bigg[\frac{K_{eq}}{Q}\bigg]
\end{equation}

If $K_{eq}/Q<1$ the reaction proceeds towards the reactants, which we can call the *negative* direction. While, if $K_{eq}/Q>1$ the reaction proceeds towards the products, *i.e.* in the positive direction.
You can indeed see that because the *driving force* is defined as the logarithm of the $K_{eq}/Q$ it correctly changes sign, *i.e.* direction, when $Q$ become smaller or larger than $K_{eq}$.

3. compute the new concentrations after a *timestep* has passed.
We now assume that for an arbitrarily short time interval the *driving force* doesn't change and compute the concentrations of all the species in the system aftern that small amount of time has passed, which corresponds to the $x$ in the *ICE* table above.

\begin{equation}
x = \delta\ln\bigg[\frac{K_{eq}}{Q}\bigg]
\end{equation}

There is no unique way to compute x, the only requirement being that it should be a comparatively small change in the system composition, otherwise the procedure will become unstable. This is because our assumption that the *driving force* will break down. In the formula above we have introduced a new parameter $\delta$ that will allow us to control how much the concentrations can change before we recompute the *driving force*.
A reasonable choice for delta could be a value around 20% of the smallest concentration we have in the system. 
The Larger values of $\delta$ the faster our procedure will converge, until the calculation becomes unstable and the method will fail to converge.
On the contrary, small values of $\delta$ will always converge to the correct solutions, but it may take a longer number of cycles.

4. Repeat from step 1 until there are no more change in the concentrations of all the species.

Follow now the demonstrator explaining how to create an excel spreadsheet that implements those steps.

5. The calculation has converged when the concentration of the species don't change anymore, *i.e.* the *driving force* is zero and the reaction quotient is equal to the equilibrium constant.

Now try to solve this proble yourself using an excel spreadsheet.
This python program can be used to verify your result.

### Important note on the first cycle:
In some cases one (or more) of the species involved in the reaction may have a zero initial concentration.
Therefore, the calculation of the reaction quotient would give $Q=0$ or $Q=\infty$, which makes the calculation of the force, $\ln\ [K_{eq}Q]$, impossible. In order to circumvent that problem, you can perform a "manual" first step of the minimisation cycle using an arbitrary (small) value for the force; *e.g.* $F=1$ or smaller. If a reactant has a zero concentration, you would have to use a small negative force.
Henceforth, when the concentration of all the species is different from zero and positive, you can follow the procedure outlined above.

- Click `Download CSV` to export the data as a CSV file to verify your result.


In [ ]:
def initialise():
    global nPoints
    global concA, concB
    global Keq
    global delta

    nPoints = 20
    concA = 1
    concB = 0.1
    Keq = 0.156
    delta = 0.2

def addLine(t,x,y,res,Q):
    var_list = []
    var_list.append(t)
    var_list.append(x)
    var_list.append(y)
    var_list.append(Q)
    res.loc[len(res)] = var_list

initialise()


In [ ]:
respath = os.path.join(os.getcwd(), "..", "results.csv")

out_P = ipw.Output()
out_L = ipw.Output()

with out_L:
    display(Markdown("[Download CSV](../results.csv)"))

    
def force(Q,k):
    if (abs(Q) > 1.e-6):
        force = - math.log(Q/k)
    else:
        force = 1.

    return force

def calc1(btn):
    out_P.clear_output()
    
    if os.path.exists(respath):
        os.remove(respath)
    res = pd.DataFrame(columns=["step" , "[A]" , "[B]", "Q"])

    A = float(concA_text.value)
    B = float(concB_text.value)
    dx = float(delta_text.value)
    k = float(Keq_text.value)
    n = int(nPoints_text.value)

    Q = B / math.pow(A,2)
    addLine(0,A,B,res,Q)

    for i in range(0, n):
        f = force(Q,k)
        cc = min(A,B)
        A = A - 2 * dx * f * cc
        B = B + dx * f * cc
        
        Q = B / math.pow(A,2)
        addLine(i,A,B,res,Q)
        # Append result

        
    res.to_csv(respath, index=False)
    with out_P:
        display(res.tail(n))

btn_calc1 = ipw.Button(description="Get Data", layout=ipw.Layout(width="150px"))
btn_calc1.on_click(calc1)

rows = []

# Equilibrium constant
Keq_text = ipw.Text(str(Keq))

# Initial concentrations
concA_text = ipw.Text(str(concA))

concB_text = ipw.Text(str(concB))

# delta concentration
delta_text = ipw.Text(str(delta))


# Nmber of data points
nPoints_text = ipw.Text(str(nPoints))

rows.append(ipw.HBox([ipw.Label('Initial concentration of A   :  '),concA_text]))
rows.append(ipw.HBox([ipw.Label('Initial concentration of B   :  '),concB_text]))
rows.append(ipw.HBox([ipw.Label('Equilibrium constant         :  '),Keq_text]))
rows.append(ipw.HBox([ipw.Label('Delta concentration          :  '),delta_text]))
rows.append(ipw.HBox([ipw.Label('Number of data point required:  '),nPoints_text]))

rows.append(ipw.HBox([btn_calc1]))

rows.append(ipw.HBox([out_L]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)